In [1]:
!pip install datasets
!pip install datasets sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

Dataset Address  [link text](#https://huggingface.co/datasets/gopalkalpande/bbc-news-summary)

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util

# Load dataset
dataset = load_dataset("gopalkalpande/bbc-news-summary")['train']

# Initialize model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode summaries (consider checking if "Summaries" is available)
passage_embeddings = model.encode([summary for summary in dataset["Summaries"] if summary], show_progress_bar=True, convert_to_tensor=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['File_path', 'Articles', 'Summaries'],
        num_rows: 2224
    })
})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/70 [00:00<?, ?it/s]

In [29]:
# Function to find relevant news articles based on a prompt
def find_relevant_news(prompt, top_k=3):
    """
    Finds and returns the top_k relevant news summaries based on the given prompt.

    Parameters:
    - prompt (str): The subject or query to search for relevant news.
    - top_k (int): Number of top relevant articles to return.

    Returns:
    - List of top_k relevant news summaries.
    """
    # Encode the prompt into an embedding
    prompt_embedding = model.encode(prompt, convert_to_tensor=True)

    # Calculate cosine similarities between the prompt and all news summaries
    similarities = util.cos_sim(prompt_embedding, passage_embeddings)

    # Find the indices of the top_k most similar news summaries
    top_indices = similarities.topk(k=top_k).indices.squeeze()

    # Extract and return the top_k relevant news summaries
    return [dataset["Summaries"][index][:200] + "." for index in top_indices]

# Example usage
find_relevant_news("latest football math of championsship")

['Arsenal go into the game on the back of a 2-0 victory over Fulham on Sunday."We do not play a season with 11 players and I believe that all of our squad deserve a chance in the team."But it is more en.',
 'Victory over Premiership strugglers Southampton, would bring the mighty Manchester United to Griffin Park and a gigantic pay day for the sole League One survivors.- To get to within two matches of a s.',
 'Indeed, it seems like there has been a CM game for as many years as there have been PCs.Eidos announced the next CM game, with a new team to develop it from scratch, whilst SI developed the existing c.']